In [ ]:
import pickle
from tkinter import *
from tkinter import ttk
from tkinter import font
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from nbclient import NotebookClient
from nbformat import read
import threading


def confirm_dataset():
    """Save the dataset selection to a pickle file"""
    def on_entry_click(event):
            """Clears the placeholder text when the entry is clicked."""
            if filepath_entry.get().strip() == "Enter Filepath":
                filepath_entry.delete(0, END)
                filepath_entry.config(foreground="black")

    def on_focus_out(event):
        """Restores placeholder text if the entry is left empty."""
        if not filepath_entry.get().strip():
            filepath_entry.insert(0, "Enter Filepath")
            filepath_entry.config(foreground="grey")
    
    global submit_entry_btn
    
    dataset = str(selected_dataset.get())  # get selected dataset from dropdown menu
    
    if dataset == "Upload Dataset":
        for widget in sub_frame.winfo_children():
            widget.destroy()  # clear widgets from sub frame

        filler_label = ttk.Label(sub_frame, text="")  # filler label for spacing
        filler_label.pack()

        filepath_entry = ttk.Entry(sub_frame, width=50, foreground="grey")
        filepath_entry.insert(0, "Enter Filepath")
        filepath_entry.pack()

        # Bind events to the entry field
        filepath_entry.bind("<FocusIn>", on_entry_click)
        filepath_entry.bind("<FocusOut>", on_focus_out)

        # Button to submit filepath
        submit_entry_btn = ttk.Button(sub_frame, text="Submit", command=lambda: submit_upload(filepath_entry))  # button to submit filepath
        submit_entry_btn.pack()
        

    elif dataset == "US Election Tweets":
        for widget in sub_frame.winfo_children():
            widget.destroy()  # remove all widgets from sub frame

        filepath = r"001215283-FYP_Code\2024_US_Election_Tweets_500.csv"
        
        with open("data.pkl", "wb") as f:
            pickle.dump(filepath, f)  # serialise filepath
       
        print("US Election Tweets")
    elif dataset == 'Israel Palastine Conflict Comments':
        for widget in sub_frame.winfo_children():
            widget.destroy()  # remove all widgets from sub frame

        filepath = r"001215283-FYP_Code\ISR_PSE_500.csv"
        
        with open("data.pkl", "wb") as f:
            pickle.dump(filepath, f)  # serialise filepath
        
        print("Israel Palestine Conflict Comments")
    elif dataset == 'Climate Change Comments':
        for widget in sub_frame.winfo_children():
            widget.destroy()  # remove all widgets from sub frame

        filepath = r"001215283-FYP_Code\reddit_climate_change_500.csv"
        
        with open("data.pkl", "wb") as f:
            pickle.dump(filepath, f)  # serialise filepath
        
        print("Climate Change Comments")
    
    run_backend()


def submit_upload(entry):
    """Submit the uploaded file and save the filepath to a pickle file"""
    global check_status_btn

    filepath = str(entry.get())  # get filepath from entry box
    with open("data.pkl", "wb") as f:
        pickle.dump(filepath, f)  # serialise filepath
    
    for widget in sub_frame.winfo_children():
        if isinstance(widget, ttk.Button) and widget != submit_entry_btn:
            widget.destroy()
    check_status_btn = ttk.Button(sub_frame, text="Check Upload Status", command=check_status)  # button to submit filepath
    check_status_btn.pack()

    run_backend()


def check_status():
    """Check the status of the uploaded file"""
    with open("file_error.pkl", "rb") as f:
        upload_status = pickle.load(f)  # load file error status
    
    for widget in sub_frame.winfo_children():
        if isinstance(widget, ttk.Label):
            widget.destroy()
    upload_status_label = ttk.Label(sub_frame, text=f"File Status: {upload_status}", font=('Arial'))  # label for file error
    upload_status_label.pack(side=BOTTOM)  # set label position    



def run_backend():
    """Run the backend code and display the results"""

    #  ---------- CLEAR OLD DATA ----------
    open("cd.pkl", "w").close()
    open("cs.pkl", "w").close()
    open("fig.pkl", "w").close()
    open("file_error.pkl", "w").close()
    open("summaries.pkl", "w").close()

    for widget in fig_frame.winfo_children():  # remove all widgets from fig frame
        widget.destroy()
    fig_show = ttk.Button(fig_frame, text="Show Figure", command=show_fig)  # button to show figure
    fig_show.pack()

    for widget in text_frame.winfo_children():  # remove all widgets from left frame
        widget.destroy()
    summary_show =  ttk.Button(text_frame, text="Show Summaries", command=print_summaries)   # button to show summaries
    summary_show.pack()
    
    def execute_backend():
        """Execute the backend code"""
        with open("Contrameter Backend.ipynb") as f:  # open backend notebook
            nb = read(f, as_version=4)
            client = NotebookClient(nb) 
            client.execute()  # run backend notebook
        
    thread = threading.Thread(target=execute_backend)
    thread.start()  # run in a separate thread
        
    
def show_fig():
    """Display the clustering figure in the GUI"""
    for widget in fig_frame.winfo_children():  # remove all widgets from fig frame
        widget.destroy()
    fig_show = ttk.Button(fig_frame, text="Show Figure", command=show_fig)  # button to show figure
    fig_show.pack()
    
    with open("fig.pkl", "rb") as f:
        fig = pickle.load(f)  # load stored figure
        
    canvas = FigureCanvasTkAgg(fig, master=fig_frame)  # construct canvas object
    canvas.draw()  # draw canvas
    canvas.get_tk_widget().pack()  # display canvas
    
    with open("cs.pkl", "rb") as f:
        controversy_score = pickle.load(f)
    cs_label = ttk.Label(fig_frame, text=f"Controversy Score: {controversy_score}", font=('Arial', 20))  # display controversy score
    cs_label.pack()

    with open("cd.pkl", "rb") as f:
        sentiment_distribution = pickle.load(f)
    cd_label = ttk.Label(fig_frame, text=f"Sentiment Distribution: {sentiment_distribution}", font=('Arial', 20))  # display cluster distribution
    cd_label.pack()
        
    
def print_summaries():
    """Output the summaries to the GUI"""
    for widget in text_frame.winfo_children():  # remove all widgets from left frame
        widget.destroy()

    summary_show =  ttk.Button(text_frame, text="Show Summaries", command=print_summaries)   # button to show summaries
    summary_show.pack()
    
    with open("summaries.pkl", "rb") as f:
        summaries = pickle.load(f)

    summary_font=font.Font(family="Arial", size=12)    
    summary_box = Text(text_frame, height=30, width=80, font=summary_font)
    summary_box.pack()
    
    for i in range(len(summaries)):
        [summary_box.insert(END, f'Cluster {i+1} Summary:\n')]
        summary_box.insert(END, summaries[i])
        summary_box.insert(END,'\n--------------------------------------------------------------------------\n')
        
        
gui = Tk()  # construct GUI window
gui.title("Contrameter")  # set window title
gui.geometry("1200x800")  # set window size

# ---------- CONSTRUCT FRAMES ----------
top_frame = ttk.Frame(gui, width=1200, height=100)  # frame for dataset selection
top_frame.pack(side=TOP)  # set frame position

sub_frame = ttk.Frame(gui, width=1200, height=100)  # frame for subframes
sub_frame.pack(side=TOP)  # set frame position

left_frame = ttk.Frame(gui, width=600, height=200)  # frame for buttons and labels
left_frame.pack(side=LEFT)  # set frame position

fig_frame = ttk.Frame(left_frame, width=600, height=400)  # frame for figure
fig_frame.pack(side=BOTTOM)

right_frame = ttk.Frame(gui, width=600, height=200)  # frame for figure
right_frame.pack(side=RIGHT)

text_frame = ttk.Frame(right_frame, width=600, height=400)  # frame for text
text_frame.pack(side=BOTTOM)


# ---------- DATASET SELECTION BUTTONS ----------
dataset_select_label = ttk.Label(top_frame, text="Contrameter", font=('Arial', 30))  # label for dataset selection
dataset_select_label.pack()  # set label position

selected_dataset = StringVar(gui)  # variable for dataset selection
options = ['Select Dataset or Upload', 'Upload Dataset', 'US Election Tweets', 'Israel Palastine Conflict Comments', 'Climate Change Comments']  # dataset options

dataset_select = ttk.OptionMenu(top_frame, selected_dataset, *options)  # dropdown menu for dataset selection
dataset_select.pack()  # set dropdown menu position

dataset_confirm = ttk.Button(top_frame, text="Confirm", command=confirm_dataset)  # button to confirm dataset selection
dataset_confirm.pack()  # set button position


gui.mainloop()


US Election Tweets


Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\nikik\anaconda3\envs\TFenv\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\nikik\AppData\Local\Temp\ipykernel_5588\2129997552.py", line 152, in show_fig
    fig = pickle.load(f)  # load stored figure
EOFError: Ran out of input
Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\nikik\anaconda3\envs\TFenv\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\nikik\AppData\Local\Temp\ipykernel_5588\2129997552.py", line 152, in show_fig
    fig = pickle.load(f)  # load stored figure
EOFError: Ran out of input
Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\nikik\anaconda3\envs\TFenv\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\nikik\AppData\Local\Temp\ipykernel_5588\2129997552.py", line 152, in show_fig
    fig =